In [1]:
import pyomo.environ as pyo

# Create the Pyomo model
m = pyo.ConcreteModel()

# Define sets
nodes = ['1', '2', '3', '4']
links = [('1', '2'), ('2', '3'), ('3', '4'), ('4', '1')]
od_pairs = [('1', '3'), ('2', '4')]
m.nodes = pyo.Set(initialize=nodes)
m.links = pyo.Set(initialize=links)
m.od_pairs = pyo.Set(initialize=od_pairs)

# Parameters
free_flow_time = {('1', '2'): 10, ('2', '3'): 15, ('3', '4'): 20, ('4', '1'): 25}
capacity = {('1', '2'): 100, ('2', '3'): 150, ('3', '4'): 120, ('4', '1'): 130}
alpha = 0.15
beta = 4
demand = {('1', '3'): 200, ('2', '4'): 150}

# Decision Variables
m.x = pyo.Var(m.links, m.od_pairs, within=pyo.NonNegativeReals)  # Flow per OD pair
m.f = pyo.Var(m.links, within=pyo.NonNegativeReals)              # Total flow on each link
m.t = pyo.Var(m.links, within=pyo.NonNegativeReals)              # Travel time on each link

# BPR Function Constraints
def bpr_function_constraint(m, i, j):
    t0 = free_flow_time[(i, j)]
    Ca = capacity[(i, j)]
    return m.t[(i, j)] == t0 * (1 + alpha * (m.f[(i, j)] / Ca)**beta)

m.bpr_function = pyo.Constraint(m.links, rule=bpr_function_constraint)

# Total flow calculation constraint
def total_flow_constraint(m, i, j):
    return m.f[(i, j)] == sum(m.x[(i, j), k] for k in m.od_pairs)

m.total_flow_constraint = pyo.Constraint(m.links, rule=total_flow_constraint)

# Objective function using the analytical integral of the BPR function
def objective_rule(m):
    total_cost = 0
    for (i, j) in m.links:
        t0 = free_flow_time[(i, j)]
        Ca = capacity[(i, j)]
        x = m.f[(i, j)]
        # Analytical integral of the BPR function
        integral_value = t0 * (x + (alpha * x**(beta + 1)) / (Ca**beta * (beta + 1)))
        total_cost += integral_value
    return total_cost

m.obj = pyo.Objective(rule=objective_rule, sense=pyo.minimize)

def flow_conservation_rule(m, od_pair_node):
    k, n = od_pair_node  # Unpacking the tuple into OD pair and node
    origin, destination = k
    
    inflow = sum(m.x[(i, n), k] for i in m.nodes if (i, n) in m.links)
    outflow = sum(m.x[(n, j), k] for j in m.nodes if (n, j) in m.links)
    
    if n == origin:
        return outflow - inflow == demand[k]  # Source node
    elif n == destination:
        return inflow - outflow == demand[k]  # Sink node
    else:
        return inflow == outflow  # Intermediate nodes


# Apply the flow conservation constraint
m.flow_conservation = pyo.Constraint(m.od_pairs * m.nodes, rule=flow_conservation_rule)



# Solve the model
solver = pyo.SolverFactory('ipopt')
result = solver.solve(m, tee=True)

# Output results
print("Optimal Flows on Links per OD Pair:")
for (i, j) in m.links:
    for k in m.od_pairs:
        print(f"Flow on link ({i}, {j}) for OD pair {k}: {pyo.value(m.x[(i, j), k]):.2f} units")

print("\nTotal Link Flows:")
for (i, j) in m.links:
    print(f"Total flow on link ({i, j}): {pyo.value(m.f[(i, j)]):.2f} units")

print("\nTravel Times on Links:")
for (i, j) in m.links:
    print(f"Travel time on link ({i}, {j}): {pyo.value(m.t[(i, j)]):.2f} minutes")

# Calculate total travel time using the optimal flows
total_travel_time = sum(
    pyo.value(m.t[(i, j)]) * pyo.value(m.f[(i, j)]) for (i, j) in m.links
)
print(f"\nTotal Travel Time: {total_travel_time:.2f}")


ERROR: Rule failed when generating expression for Constraint flow_conservation
with index ('1', '3', '1'): TypeError: flow_conservation_rule() takes 2
positional arguments but 4 were given
ERROR: Constructing component 'flow_conservation' from data=None failed:
        TypeError: flow_conservation_rule() takes 2 positional arguments but 4
        were given


TypeError: flow_conservation_rule() takes 2 positional arguments but 4 were given

In [ ]:
print(pyo.value())